In [ ]:
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis.constants import SPATIAL, NODATAINT, MAX, SQUARE, EXACT
from geopyspark.geotrellis.rdd import TiledRasterRDD

import json
import shapely
from shapely.geometry import shape, asShape, MultiPoint, MultiLineString

In [ ]:
geopysc = GeoPyContext(appName="libya", master="local[*]")

In [ ]:
from geopyspark.geotrellis.tms import make_s3_tms, remote_tms_server
import geopyspark.geotrellis.color as Color

watercolor = remote_tms_server(geopysc, "http://tile.stamen.com/watercolor/{z}/{x}/{y}.jpg")
server = make_s3_tms(geopysc, "azavea-datahub", "catalog", "nlcd-tms-epsg3857", Color.ColorMap.nlcd_colormap(geopysc))

In [ ]:
from geonotebook.wrappers.raster import TMSRasterData

In [ ]:
M.add_layer(TMSRasterData(watercolor), name="watercolor")

In [ ]:
M.add_layer(TMSRasterData(server), name="TMS")

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
from geopyspark.geotrellis.tms import remote_tms_server

watercolor = remote_tms_server(geopysc, "http://tile.stamen.com/watercolor/{z}/{x}/{y}.jpg")